In [71]:
from abc import ABC, abstractmethod


Define our objects that will be used for our rules processing.


In [72]:
class JobAttribute(object):
    def __init__(self, id: int, name: str, value: str | list[str] | int | float | bool):
        self._id = id
        self._name = name
        self._value = value

    @property
    def name(self):
        return self._name

    @property
    def id(self):
        return self._id

    @property
    def attr_value(self):
        return self._value


class Job(object):
    def __init__(self, external_job_id: int, segment_id: int, source_system_id: int):
        self.external_job_id = external_job_id
        self.segment_id = segment_id
        self.source_system_id = source_system_id
        self.job_attributes: list[JobAttribute] = []

    def add_attribute(self, id: int, name: str, value: int | float | str | bool):
        ja = JobAttribute(id, name, value)
        self.job_attributes.append(ja)

    def get_attribute(self, id) -> JobAttribute:
        attr: JobAttribute
        for attr in self.job_attributes:
            if attr.id == id:
                return attr


Define our rules. All rules will inherit from the abstract Specification class so that they implement the "is_satisfied_by" method, which always returns a boolean value


In [73]:
class Specification(ABC):
    @abstractmethod
    def is_satisfied_by(self, item: Job) -> bool:
        pass


class RangeRule(Specification):
    def __init__(
        self, attribute_id: int, min_value: int | float, max_value: int | float
    ):
        self.attribute_id = attribute_id
        self.min_value: int | float = min_value
        self.max_value: int | float = max_value

    def is_satisfied_by(self, job) -> bool:
        attr = job.get_attribute(self.attribute_id)
        return (
            attr
            and attr.attr_value >= self.min_value
            and attr.attr_value <= self.max_value
        )


class SkillLevelRule(Specification):
    def __init__(self, skill_level: int):
        self.skill_level = skill_level

    def is_satisfied_by(self, job: Job) -> bool:
        # here, we know that the attribute id is always 87
        # and the caller does not need to know that.
        attr = job.get_attribute(87)
        return attr and attr.attr_value >= self.skill_level
    
    
class AssetCodeNameRule(Specification):
    def __init__(self, acceptable_codes: list[str]):
        self.acceptable_codes = acceptable_codes
        
    def is_satisfied_by(self, job: Job) -> bool:
        attr = job.get_attribute(873)
        code: str = attr._value
        return code in self.acceptable_codes


This method will accept a job and an arbitrary number of rules to apply.  If any of the rules evaluate to false, then the whole check is false.

In [74]:
def check_rules(job: Job, *specs) -> bool:
    spec: Specification
    for spec in specs:
        if not spec.is_satisfied_by(job):
            return False
    return True


Setup the expectations for this job.

In [75]:
# Surface tension must be between 3400 and 3500 
surfaceTensionCheck = RangeRule(5, 3400, 3500)
# The people on this job must have a skill level of 2 or greater
skillCheck = SkillLevelRule(2)
# Acceptable Codes
codeCheck = AssetCodeNameRule(["XYZ", "BR549", "C2255", "DZ-015", "27B-6"])

In [77]:
job = Job(123, 45, 678)
job.add_attribute(5, "SurfaceTension", 3456)
job.add_attribute(87, "SkillLevel", 3)
job.add_attribute(873, "AssetCodeName", "DZ-015")

check_rules(job, surfaceTensionCheck, skillCheck, codeCheck)


False